In [64]:
!pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
import requests
import http.client
from bs4 import BeautifulSoup
import pandas as pd
import re
import traceback
import yfinance as yf
from datetime import datetime
from matplotlib import pyplot as plt

year1 = 'Mar-2022'
year2 = 'Mar-2021'
year3 = 'Mar-2020'
year4 = 'Mar-2019'

#These below categories are analysed 
LargeCapMar2022 = {}
MidCapMar2022 = {}
SmallCapMar2022 = {}

LargeCapMar2021 = {}
MidCapMar2021 = {}
SmallCapMar2021 = {}

LargeCapMar2020 = {}
MidCapMar2020 = {}
SmallCapMar2020 = {}

LargeCapMar2019 = {}
MidCapMar2019 = {}
SmallCapMar2019 = {}


# We have picked up the 30 stocks listed in NSE index and chosen based on their top marketcap. The details are mentioned on the Google Sheet 


largeCapStocks = ['BAJAJ-AUTO.NS',	'TATASTEEL.NS',	'ICICIBANK.NS',	'HINDALCO.NS',	'ONGC.NS',	'BRITANNIA.NS',	'MARUTI.NS',	
          'INDUSINDBK.NS',	'NESTLEIND.NS',	'NTPC.NS',	'COALINDIA.NS',	'ADANIENT.NS',	'ITC.NS',	'WIPRO.NS',	'TITAN.NS',
          'RELIANCE.NS',	'KOTAKBANK.NS',	'TCS.NS',	'HEROMOTOCO.NS',	'CIPLA.NS']

midCapStocks = ['TRIDENT.NS',	'KPITTECH.NS',	'AAVAS.NS',	'ISEC.NS',	'AJANTPHARM.NS',	'DCMSHRIRAM.NS',	'IIFLWAM.NS',	'BDL.NS',	'CREDITACC.NS',	'APTUS.NS',	'JBCHEPHARM.NS',
                'NH.NS',	'ASAHIINDIA.NS',	'GODREJIND.NS',	'RATNAMANI.NS',	'NUVOCO.NS',	'HAPPSTMNDS.NS',	'NIACL.NS',	'CUB.NS',	'TTKPRESTIG.NS']


smallCapStocks = ['IDBI.NS',	'METROBRAND.NS',	'TIMKEN.NS',	'KPITTECH.NS',	'APOLLOTYRE.NS',	'CENTRALBK.NS',	'CAMPUS.NS',	'FINEORG.NS',	
                  'IIFL.NS',	'DCMSHRIRAM.NS',	'APTUS.NS',	'CARBORUNIV.NS',	'IIFLWAM.NS',	'JBCHEPHARM.NS',
                  'ELGIEQUIP.NS',	'CREDITACC.NS',	'ASAHIINDIA.NS',	'IRB.NS',	'NUVOCO.NS']







In [ ]:
#Scenario Configuration

stocks = largeCapStocks
scenarioTest = 'LargeCapMar2022'
year = year1

In [67]:
# Dictionary to hold information for each stock/ticker
financial_dir = {}

for ticker in stocks:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://finance.yahoo.com/quote/'+ticker+'/balance-sheet'
        headerFake = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        page = requests.get(url,headers=headerFake)
        page_content = page.content
        #soup = BeautifulSoup(page_content,'html.parser')
        soup = BeautifulSoup(page_content, 'html5lib')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "Va(m)"})
            for row in rows:
                temp_dir[row.get_text()] = soup.find('span', string=row.get_text()).find_next('span').text            
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url,headers=headerFake)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
           rows = t.find_all("div", {"class" : "Va(m)"})
           for row in rows:
             temp_dir[row.get_text()] = soup.find('span', string=row.get_text()).find_next('span').text
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url,headers=headerFake)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "Va(m)"})
            for row in rows:
                temp_dir[row.get_text()] = soup.find('span', string=row.get_text()).find_next('span').text
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url,headers=headerFake)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html5lib')
        temp_dir['Market Cap (intra-day)'] = soup.find('span',string='Market Cap (intraday)' ).find_next('td').text
        temp_dir['Forward Annual Dividend Yield'] = soup.find('span',string='Forward Annual Dividend Yield').find_next('td').text         

        #getting stocks return between two dates
          #getting stocks return between two dates
        if year == 'Mar-2022':
          data1 = yf.download(ticker, start='2022-04-01', end='2022-04-02', progress=False).to_numpy()
          data2 = yf.download(ticker, start='2022-10-28', end='2022-10-31', progress=False).to_numpy()
          retunYearly = (data2[0,1]-data1[0,1])/data1[0,1]
          temp_dir['returns'] = "{:.1%}".format(retunYearly)
        elif year =='Mar-2021':
          data1 = yf.download(ticker, start='2021-04-01', end='2021-04-02', progress=False).to_numpy()
          data2 = yf.download(ticker, start='2022-03-30', end='2022-03-31', progress=False).to_numpy()
          retunYearly = (data2[0,1]-data1[0,1])/data1[0,1]
          temp_dir['returns'] = "{:.1%}".format(retunYearly)  
        elif year =='Mar-2020':
          data1 = yf.download(ticker, start='2020-04-01', end='2020-04-02', progress=False).to_numpy()
          data2 = yf.download(ticker, start='2021-03-30', end='2021-03-31', progress=False).to_numpy()
          retunYearly = (data2[0,1]-data1[0,1])/data1[0,1]
          temp_dir['returns'] = "{:.1%}".format(retunYearly) 
        elif year =='Mar-2019':
          data1 = yf.download(ticker, start='2019-04-01', end='2019-04-02', progress=False).to_numpy()
          data2 = yf.download(ticker, start='2020-03-30', end='2020-03-31', progress=False).to_numpy()
          retunYearly = (data2[0,1]-data1[0,1])/data1[0,1]
          temp_dir['returns'] = "{:.1%}".format(retunYearly)     

        #PPE Value
        ticker_object = yf.Ticker(ticker)
        df = ticker_object.balancesheet
        for i, j in df.iterrows():
          if(i=='Property Plant Equipment'):
            temp_dir[i] = j[0];
        #combining all extracted information with the corresponding ticker into the dictionary
        financial_dir[ticker] = temp_dir
        print("Data successfully scraped for ", ticker)
    except:
        print("Problem scraping data for ",ticker)
print('All data webscraped')

Data successfully scraped for  BAJAJ-AUTO.NS
Data successfully scraped for  TATASTEEL.NS
Data successfully scraped for  ICICIBANK.NS
Data successfully scraped for  HINDALCO.NS
Data successfully scraped for  ONGC.NS
Data successfully scraped for  BRITANNIA.NS
Data successfully scraped for  MARUTI.NS
Data successfully scraped for  INDUSINDBK.NS
Data successfully scraped for  NESTLEIND.NS
Data successfully scraped for  NTPC.NS
Data successfully scraped for  COALINDIA.NS
Data successfully scraped for  ADANIENT.NS
Data successfully scraped for  ITC.NS
Data successfully scraped for  WIPRO.NS
Data successfully scraped for  TITAN.NS
Data successfully scraped for  RELIANCE.NS
Data successfully scraped for  KOTAKBANK.NS
Data successfully scraped for  TCS.NS
Data successfully scraped for  HEROMOTOCO.NS
Data successfully scraped for  CIPLA.NS
Data successfully scraped for  APOLLOHOSP.NS
Data successfully scraped for  TATACONSUM.NS
Data successfully scraped for  TECHM.NS
Data successfully scraped f

In [68]:
# Storing the scraped info to a Dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all Null values
tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted

for ticker in stocks:
   try:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]
   except Exception as e:
     print(e)

'MM.NS'


In [76]:
combined_financials.head()
financial_dir

{'BAJAJ-AUTO.NS': {'Total Assets': '351,112,100',
  'Total Liabilities Net Minority Interest': '52,515,500',
  'Total Equity Gross Minority Interest': '298,596,600',
  'Total Capitalization': '298,596,500',
  'Common Stock Equity': '298,596,500',
  'Net Tangible Assets': '298,343,300',
  'Working Capital': '56,771,100',
  'Invested Capital': '298,596,500',
  'Tangible Book Value': '298,343,300',
  'Total Debt': '1,245,200',
  'Share Issued': '289,367',
  'Ordinary Shares Number': '289,367',
  'Total Revenue': '343,123,900',
  'Cost of Revenue': '264,032,000',
  'Gross Profit': '79,091,900',
  'Operating Expense': '21,191,000',
  'Operating Income': '57,900,900',
  'Net Non Operating Interest Income Expense': '8,674,800',
  'Pretax Income': '74,798,700',
  'Tax Provision': '16,412,600',
  'Net Income Common Stockholders': '58,386,100',
  'Diluted NI Available to Com Stockholders': '58,386,100',
  'Basic EPS': 'Diluted EPS',
  'Diluted EPS': 'Basic Average Shares',
  'Basic Average Share

In [70]:
# Required stats for the magic formula
stats = ["EBITDA",
         "Reconciled Depreciation",
         "Market Cap (intra-day)",
         "Net Income Common Stockholders",
         "Operating Cash Flow",
         "Capital Expenditure",
         "Total Assets",
         "Total Liabilities Net Minority Interest",
         "Property Plant Equipment",
         "Total Equity Gross Minority Interest",
         "Total Debt",
         "Forward Annual Dividend Yield","returns"]

# Short hands for above to keep things simple
indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield","returns"]

In [71]:
all_stats = {}
for ticker in stocks:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except Exception as e:
        print("Can't read data for ",ticker)


Can't read data for  MM.NS


In [72]:
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)

stocks = all_stats_df.columns

In [73]:
all_stats_df


,BAJAJ-AUTO.NS,TATASTEEL.NS,HINDALCO.NS,ONGC.NS,MARUTI.NS
EBITDA,7.792310e+07,6.393838e+08,2.435700e+08,6.060076e+08,8.219900e+07
D&A,2.748400e+06,9.013120e+07,6.733000e+07,2.688316e+08,2.697200e+07
MarketCap,1.040000e+09,1.240000e+09,9.033900e+08,1.680000e+09,2.870000e+09
NetIncome,5.838610e+07,3.901085e+08,1.506200e+08,5.147354e+08,4.440700e+07
CashFlowOps,4.232410e+07,4.438099e+08,1.683800e+08,7.800004e+08,1.840500e+07
Capex,-7.961500e+06,-1.052220e+08,-5.426000e+07,-4.452634e+08,-3.459300e+07
CurrAsset,3.511121e+08,2.854456e+09,2.230620e+09,2.784233e+09,7.465550e+08
CurrLiab,5.251550e+07,1.683471e+09,1.448600e+09,2.784233e+09,1.932200e+08
PPE,1.914420e+10,1.457318e+12,8.119700e+11,3.136458e+12,1.604350e+11
BookValue,2.985966e+08,1.170985e+09,7.820200e+08,2.833278e+09,5.533350e+08


In [74]:
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]


In [75]:
final_stats_val_df = final_stats_df.loc[stocks,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
value_stocks['Returns'] = transpose_df["returns"]

print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")

print(value_stocks)
value_stocks.plot()
if(scenarioTest=='LargeCapMar2022'):
  LargeCapMar2022 = value_stocks
elif(scenarioTest=='MidCapMar2022'):
  MidCapMar2022 = value_stocks
elif(scenarioTest=='SmallCapMar2022'):
  SmallCapMar2022 = value_stocks
elif(scenarioTest=='LargeCapMar2021'):
  LargeCapMar2021 = value_stocks
elif(scenarioTest=='MidCapMar2021'):
  MidCapMar2021 = value_stocks
elif(scenarioTest=='SmallCapMar2021'):
  SmallCapMar2021 = value_stocks
elif(scenarioTest=='LargeCapMar2020'):
  LargeCapMar2020 = value_stocks
elif(scenarioTest=='MidCapMar2020'):
  MidCapMar2020 = value_stocks
elif(scenarioTest=='SmallCapMar2020'):
  SmallCapMar2020 = value_stocks




------------------------------------------------
Value stocks based on Greenblatt's Magic Formula
               EarningYield       ROC  MagicFormulaRank  Returns
TATASTEEL.NS       0.666120  0.000377               1.0   -0.220
BAJAJ-AUTO.NS      0.101225  0.003866               2.0   -0.007
HINDALCO.NS        0.230000  0.000217               3.0   -0.285
ONGC.NS            0.116273  0.000108               4.0   -0.194
MARUTI.NS          0.023795  0.000343               5.0    0.236


In [78]:
print('Mean Value for top 10 Large Cap Magic Formula Return ' + str(value_stocks.iloc[4].mean()))

Mean Value for top 10 Large Cap Magic Formula Return 1.3150345887524515


In [ ]:
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
high_dividend_stocks.to_frame()

In [ ]:
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
value_high_div_stocks['Returns'] = transpose_df["returns"]

print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)

#Mean Return from Magic Investment 

print('Mean Value for top 10 Large Cap Magic Formula Return ' + str(value_high_div_stocks.iloc[10].mean()))

plt.figure(figsize=(18,16))
plt.plot(value_high_div_stocks.loc[:,['EarningYield','ROC','Returns']])
